# Cyclical epidemic dynamics
Both in models and in reality, we can observe regular cycles in
the epidemiology of infectious disease outbreaks.
As we saw in notebooks [02](./02-basic-model-intro.ipynb) 
and [10](./10-reproduction-number.ipynb),
although the susceptible population is never depleted to zero,
it is the depletion in susceptibles that leads 
to the epidemic slowing, declining and dying away.
A consequence of this phenomenon is that if there is
some process of replenishing the susceptible population,
it is not inevitable that the epidemic will consistently decline.
We saw a slight suggestion of this in [notebook 06](./06-immunity.ipynb),
in which the SIRS model allowed for recovered persons
to return to the susceptible pool.
This led to a marginal resurgence in the epidemic
within the analysis period we chose.

However, if the rate of replenishment is very slow,
the epidemic may have died away to lower levels of transmission
by the time at which the susceptible population
reaches the point at which the reproduction number ($R(t)$)
increases above one and transmission begins to increase again.

So if there is some process whereby susceptibles
are replenished back into the population,
there is the potential to observe cyclical epidemics
even when model parameters remain constant.

## Waning immunity model
Let's see this in action,
first with a similar model to the one that we have been using in several previous chapters,
but with waning immunity incorporated at a very slow rate.

In [ ]:
try:
    import google.colab
    %pip install summerepi2==1.3.6
except:
    pass

In [ ]:
import pandas as pd
pd.options.plotting.backend = "plotly"
import plotly.express as px

from summer2 import CompartmentalModel
from summer2.parameters import Parameter

In [ ]:
def build_sir_model(
    config: dict,
) -> CompartmentalModel:

    compartments = (
        "susceptible", 
        "infectious", 
        "recovered",
    )
    model = CompartmentalModel(
        times=(0.0, config["end_time"]),
        compartments=compartments,
        infectious_compartments=["infectious"],
    )
    model.set_initial_population(
        distribution={
            "susceptible": config["population"] - config["seed"], 
            "infectious": config["seed"],
        },
    )
    
    model.add_infection_frequency_flow(
        name="infection", 
        contact_rate=Parameter("contact_rate"),
        source="susceptible", 
        dest="infectious",
    )
    model.add_transition_flow(
        name="recovery", 
        fractional_rate=Parameter("recovery_rate"), 
        source="infectious", 
        dest="recovered",
    )
    
    return model

In [ ]:
model_config = {
    "start_time": 0.0,
    "end_time": 1e3,
    "population": 1e5,
    "seed": 1.0,
}
sir_wane_model = build_sir_model(model_config)

sir_wane_model.add_transition_flow(
    name="waning", 
    fractional_rate=Parameter("replenishment"), 
    source="recovered", 
    dest="susceptible",
)

In [ ]:
parameters = {
    "recovery_rate": 0.333,
    "contact_rate": 1.0,
    "replenishment": 0.005,
}
population = 1e5
seed = 1.0

sir_wane_model.run(parameters=parameters)
outputs = sir_wane_model.get_outputs_df()
axis_labels = {"index": "time", "value": "compartment size"}
outputs.plot(labels=axis_labels)

As always, we encourage the user to experiment with 
different parameter configurations
to understand their effects on the cyclical dynamics.
In general, a higher contact rate will make for a lower
proportion of susceptibles needed to precipitate a new epidemic cycle,
meaning that the replenishment process will lead to a new epidemic sooner.
A faster replenishment rate will similarly trigger 
an earlier new epidemic cycle.
The pattern of an initial large epidemic wave followed by
progressively smaller waves can be referred to as "damped oscillation".

## Demographic process
Another way for susceptibles to be replenished
is to allow births into the model,
because for most infectious diseases
we would consider that babies enter the population with no immunity.
To keep this model similar to the previous one,
we'll use the parameter value we used for the waning immunity process,
but instead use it to represent the rate of death
from causes other than the infectious disease we're simulating.
We'll keep the total population size fixed ("closed") over time,
by replacing these deaths with new births into the susceptible compartment.

In [ ]:
sir_demog_model = build_sir_model(model_config)
sir_demog_model.add_universal_death_flows(
    "universal_death",
    death_rate=Parameter("replenishment"),
)
sir_demog_model.add_replacement_birth_flow(
    "births",
    "susceptible",
)

sir_demog_model.run(parameters=parameters)
sir_demog_model.get_outputs_df().plot(labels=axis_labels)

The epidemic dynamics are very similar to those observed for this model
as for when we replenished the population through waning of
immunity in the recovered population.
Note that they are not absolutely identical,
because with this "demographic" model,
the deaths are applied to all compartments.
This makes only a really marginal difference
by slightly reducing the sojourn time in the infectious compartment
and so reducing the basic reproduction number.
## Phase plane
We've seen these cyclical epidemics by looking at
the compartment sizes over time.
However, this process of oscillatory dynamics
heading gradually towards a stable endemic state
at which the recovered and the susceptible populations
balance one another can be illustrated in other ways.

An alternative way to think about what is happening in the model
is through a "phase plane" in which we plot
the susceptible and infectious populations
against one another.
This gives a nice sense of the endemic state as a 
stable equilibrium point that is attracting the epidemic
towards it at any point in time.
However, the momentum of the epidemic starts
off so great that it continually overshoots this stable point
and ends up spiralling towards it's final resting place.
You can imagine it as a bit like a coin wishing well,
with the time dimension approximately represented 
by the distance along the spiral.

In [ ]:
outputs_late_start = outputs.loc[outputs.index > 70.0]

fig = px.line(
    outputs_late_start, 
    x="susceptible", 
    y="infectious"
)
fig.show()

To explain what's happening here,
first the susceptible population is steadily increasing
along the bottom of the graph,
before the epidemic has really got going.
Next, we get the first big epidemic wave,
looping from the bottom right of the plot
across the top.
However, rather than heading straight
for the equilibrium point in the middle of the spiral,
the line of the epidemic is drawn or attracted
across to the right even as it heads in that general direction.
This leads to it overshooting the equilibrium point with each epidemic cycle,
and the epidemic gradually oscillating towards the stable equilibrium in the middle.

Another way to represent this is with a 3-D graph,
so that we can use the third dimension for time.

In [ ]:
px.line_3d(
    outputs_late_start, 
    x="susceptible", 
    y="infectious", 
    z=outputs_late_start.index,
    width=800,
    height=600,
)